In [ ]:
from pathlib import Path
import jsonlines
import pandas as pd


dump_dir = Path("mbart50_dumps")

for dump_path in dump_dir.iterdir():
    # if "backtranslation" in dump_path.name and "just_one" in dump_path.name:
    if "backtranslation" in dump_path.name and "just_one" not in dump_path.name:
        with jsonlines.open(dump_path, 'r') as reader:
            bt_results = list(reader)
        bt_results = pd.DataFrame(bt_results)
        
        score_cols = [col for col in bt_results.columns if "score" in col and "bt" not in col]
        bt_score_cols = [col for col in bt_results.columns if ("score" in col or "logprob" in col) and "bt" in col]

        all_reranked_scores = []
        for rerank_column in bt_score_cols:
            idxmax = bt_results.groupby("id")[rerank_column].idxmax()
            bt_results_reranked = bt_results.loc[idxmax]
            reranked_scores = bt_results_reranked[score_cols].mean()
            reranked_scores.name = f"reranked_{rerank_column}"
            all_reranked_scores.append(reranked_scores)
        all_reranked_scores = pd.concat(all_reranked_scores, axis=1)
        best_reranked_scores = all_reranked_scores.T.max()
        best_reranked_scores.name = "reranked_best"

        bt_results_ranked_by_confidence = bt_results.drop_duplicates("id", keep="first")
        orig_scores = bt_results_ranked_by_confidence[score_cols].mean()
        orig_scores.name = "orig"

        oracle_best_scores = {}
        oracle_worst_scores = {}
        for score_col in score_cols:
            sorted = bt_results[["id", score_col]].sort_values(["id", score_col])
            oracle_best = sorted.drop_duplicates("id", keep="last")[score_col].mean()
            oracle_worst = sorted.drop_duplicates("id", keep="first")[score_col].mean()
            oracle_best_scores[score_col] = oracle_best
            oracle_worst_scores[score_col] = oracle_worst
        oracle_best_scores = pd.DataFrame({"oracle_best": oracle_best_scores})
        oracle_worst_scores = pd.DataFrame({"oracle_worst": oracle_worst_scores})

        random_ranking_scores = bt_results[score_cols].mean()
        random_ranking_scores.name = "random"

        is_better = best_reranked_scores > orig_scores
        is_better.name = "is_better"

        all_scores = pd.concat([orig_scores, best_reranked_scores, is_better, oracle_best_scores, random_ranking_scores, oracle_worst_scores, all_reranked_scores], axis=1)

        lang_pair = [substr for substr in dump_path.name.split('_') if '-' in substr][0]
        print()
        print(lang_pair)
        display(all_scores)
